<a href="https://colab.research.google.com/github/sumukhbhat12/Tensor-Flow-Course-FreeCodeCamp/blob/main/Classification_on_Iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Includes three iris species with 50 samples each as well as some properties about each flower. One flower species is linearly separable from the other two, but the other two are not linearly separable from each other.


Dataset: https://www.kaggle.com/datasets/uciml/iris

In [81]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [82]:
data = pd.read_csv('/content/Iris.csv')

In [83]:
data

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [84]:
data = data.drop(['Id'], axis=1)

In [85]:
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


convert string classes into numbers 0,1 and 2

In [86]:
species_num = []

dic = {
    'Iris-setosa': 0,
    'Iris-versicolor': 1,
    'Iris-virginica': 2
}

for species in data['Species']:
  species_num.append(dic[species])

data['Species'] = species_num

split into train and test

In [87]:
train, test = np.split(data.sample(frac=1), [int(0.7 * len(data))])

In [88]:
train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
91,6.1,3.0,4.6,1.4,1
28,5.2,3.4,1.4,0.2,0
124,6.7,3.3,5.7,2.1,2
143,6.8,3.2,5.9,2.3,2
41,4.5,2.3,1.3,0.3,0


In [89]:
test.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
17,5.1,3.5,1.4,0.3,0
133,6.3,2.8,5.1,1.5,2
73,6.1,2.8,4.7,1.2,1
39,5.1,3.4,1.5,0.2,0
106,4.9,2.5,4.5,1.7,2


In [90]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [91]:
X_train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
91,6.1,3.0,4.6,1.4
28,5.2,3.4,1.4,0.2
124,6.7,3.3,5.7,2.1
143,6.8,3.2,5.9,2.3
41,4.5,2.3,1.3,0.3


In [92]:
y_train.head()

91     1
28     0
124    2
143    2
41     0
Name: Species, dtype: int64

Input function

In [93]:
def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

feature columns

In [94]:
feature_cols = []

for key in X_train.keys():
  feature_cols.append(tf.feature_column.numeric_column(key=key))

print(feature_cols)

[NumericColumn(key='SepalLengthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLengthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


building the model

Deep Neural Network with 2 hidden layers

In [95]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_cols,
    hidden_units=[30,10],
    n_classes=3
)

Training

In [96]:
classifier.train(
    input_fn = lambda: input_fn(X_train, y_train, training=True),    #no need to include lambda if you create a separate function to create input functions
    steps=5000
)

evaluate

In [97]:
result = classifier.evaluate(
    input_fn= lambda: input_fn(X_test, y_test, training=False)
)

In [98]:
result['accuracy']

0.95555556

prediction